In [1]:
%load_ext tensorboard

# Coppy các file cần test #

In [6]:
import shutil
import os
import json 
from tqdm import tqdm

with open("/mnt/wsl/PHYSICALDRIVE0p1/toan/Speech2Text/Test/part1/Datalabeled_18_12.json") as f:
    dataset = json.load(f)
    
for record in tqdm(dataset):
    src_path = os.path.join("/mnt/wsl/PHYSICALDRIVE0p1/toan/ASR_Label/",record['path'])
    dst_path = os.path.join("/mnt/wsl/PHYSICALDRIVE0p1/toan/Speech2Text/Test/part1",record['path'])
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copyfile(src_path,dst_path)

100%|██████████| 7465/7465 [00:00<00:00, 9893.76it/s] 


# Chạy Test #

In [2]:

from transformers import WhisperProcessor, WhisperForConditionalGeneration
import json
from evaluate import load
import os
import string
import unicodedata
import re
import librosa
from tqdm import tqdm
import shutil

MODEL_NAME = '/mnt/wsl/PHYSICALDRIVE0p1/toan/Speech2Text/Trainning/models/whisper-medium-vi-aia-27-12-2023/checkpoint-26632'
AUDIO_FOLDER = '/mnt/wsl/PHYSICALDRIVE0p1/toan/Speech2Text/RawDatasets'
JSON_FILE = "/mnt/wsl/PHYSICALDRIVE0p1/toan/Speech2Text/Datasets/Test/part1/Datalabeled_18_12.json"

def preprocess_text(text: str):
    #remove punt
    translator = str.maketrans(' ', ' ', string.punctuation)
    text = text.translate(translator)
    text = text.lower()
    text = unicodedata.normalize("NFKC", text)
    text = text.replace('\xa0', '')
    text = re.sub(r"\s{2,}", ' ',text);
    return text

class Whisper_Model:
    def __init__(self,model_name,task, language) -> None:
        self.processor = WhisperProcessor.from_pretrained("openai/whisper-small",cache_dir="./models/")
        self.model = WhisperForConditionalGeneration.from_pretrained(model_name,cache_dir="./models/")
        self.forced_decoder_ids = self.processor.get_decoder_prompt_ids(language=language, task=task)
        
    def infer(self,audiopath:str) -> str:
        y,src = librosa.load(audiopath,sr=16000,mono=True) 
        input_features = self.processor(y, sampling_rate=src, return_tensors="pt").input_features
        predicted_ids = self.model.generate(input_features, forced_decoder_ids=self.forced_decoder_ids)
        transcription = self.processor.batch_decode(predicted_ids)
        transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)
        return transcription

whisper_model = Whisper_Model(model_name= MODEL_NAME,task='transcribe',language="vi")
wer = load('wer')
with open(JSON_FILE,encoding='utf-8') as f:
    data_test = json.load(f)

bac_pre = []
bac_ref = []
trung_pre = []
trung_ref = []
nam_pre = []
nam_ref = []


for record in tqdm(data_test):
    predict = whisper_model.infer(os.path.join(AUDIO_FOLDER,record['path']))[0]
    refer = preprocess_text(record['gold_sentence'])
    record[MODEL_NAME] = predict
    if record['location'] == 'bac':
        bac_pre.append(predict)
        bac_ref.append(refer)
    elif record['location'] == 'nam':
        nam_pre.append(predict)
        nam_ref.append(refer)
    elif record['location'] == 'trung':
        trung_pre.append(predict)
        trung_ref.append(refer)

all_pre = bac_pre+trung_pre+nam_pre
all_ref = bac_ref+trung_ref+nam_ref


  2%|▏         | 118/7465 [10:44<11:06:32,  5.44s/it]

In [2]:
with open(JSON_FILE,'w',encoding='utf-8') as f:
    json.dump(data_test,f,ensure_ascii=False)

In [1]:
import json 
import string
import unicodedata
import re
from tqdm import tqdm

def preprocess_text(text: str):
    #remove punt
    translator = str.maketrans(' ', ' ', string.punctuation)
    text = text.translate(translator)
    text = text.lower()
    text = unicodedata.normalize("NFKC", text)
    text = text.replace('\xa0', '')
    text = re.sub(r"\s{2,}", ' ',text);
    return text

with open("/mnt/wsl/PHYSICALDRIVE0p1/toan/data_speech_to_text/Test/part1/Datalabeled_18_12.json",encoding='utf-8') as f:
    data_test = json.load(f)

aia_predicts = [] 
predicts = []
references = []
bac_predicts = []
aia_bac_predicts = []
bac_references = []
aia_trung_predicts = []
trung_predicts = []
trung_references = []
aia_nam_predicts = []
nam_predicts = []
nam_references = []
for record in tqdm(data_test):
    gold_sentence = preprocess_text(record['gold_sentence'])
    aia_sentence = preprocess_text(record['openai/whisper-medium'])
    whisper_sentence = preprocess_text(record['whisper_base'][0])
    if gold_sentence == "":
        continue
    aia_predicts.append(aia_sentence)
    predicts.append(whisper_sentence)
    references.append(gold_sentence)
    if record['location'] == 'bac':
        aia_bac_predicts.append(aia_sentence)
        bac_predicts.append(whisper_sentence)
        bac_references.append(gold_sentence)
    elif record['location'] == 'trung':
        aia_trung_predicts.append(aia_sentence)
        trung_predicts.append(whisper_sentence)
        trung_references.append(gold_sentence)
    elif record['location'] == 'nam':
        aia_nam_predicts.append(aia_sentence)
        nam_predicts.append(whisper_sentence)
        nam_references.append(gold_sentence)

100%|██████████| 7465/7465 [00:00<00:00, 43794.49it/s]


In [2]:
from evaluate import load
wer = load("wer")

print('====whisper-base====')
print('all wer: ',wer.compute(predictions=predicts,references=references))    
print('bac wer: ',wer.compute(predictions=bac_predicts,references=bac_references))    
print('trung wer: ',wer.compute(predictions=trung_predicts,references=trung_references))    
print('nam wer: ',wer.compute(predictions=nam_predicts,references=nam_references))
print('====aia-whisper-base====')
print('all wer: ',wer.compute(predictions=aia_predicts,references=references))    
print('bac wer: ',wer.compute(predictions=aia_bac_predicts,references=bac_references))    
print('trung wer: ',wer.compute(predictions=aia_trung_predicts,references=trung_references))    
print('nam wer: ',wer.compute(predictions=aia_nam_predicts,references=nam_references))

    

/home/nlp_team/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


====whisper-base====
all wer:  0.5086803355551613
bac wer:  0.4748821372446307
trung wer:  0.36468646864686466
nam wer:  0.5895765472312704
====aia-whisper-base====
all wer:  0.26416943339162463
bac wer:  0.25060240963855424
trung wer:  0.30115511551155116
nam wer:  0.2876612377850163
